In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

## Reflect Tables into SQLALchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

## Bonus Challenge Assignment: Temperature Analysis II

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, maximum, and average temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# For example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use the function `calc_temps` to calculate the tmin, tavg, and tmax 
# for a year in the data set
temps = calc_temps('2017-08-01', '2017-08-07')[0]
temps

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for bar height (y value)
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
lowerr = temps[1] - temps[0]
upperr = temps[2] - temps[1]

plt.figure(figsize=(3,6))
plt.bar(x='AVG', height=temps[1], yerr=[[lowerr], [upperr]], color='orange', alpha=.6)

plt.title('Trip Avg Temp')
plt.xlabel(' ')
plt.ylabel('Temp (F)')

plt.ylim(0,100)

plt.show()

### Daily Rainfall Average

In [ ]:
session.query(Measurement).first().__dict__

In [ ]:
session.query(Station).first().__dict__

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's 
# matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation
query_list = [func.avg(Measurement.prcp), Station.station, Station.name, 
              Station.latitude, Station.longitude, Station.elevation]
start = '2017-08-01'
end = '2017-08-07'
session.query(*query_list).\
    filter(Measurement.station == Station.station).\
    filter(Measurement.date >= start).filter(Measurement.date <= end).\
    group_by(Measurement.station).\
    order_by(func.avg(Measurement.prcp).desc()).all()

In [ ]:
# Use this function to calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()

# For example
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip
start_date = '2017-08-01'
end_date = '2017-08-07'

# Use the start and end date to create a range of dates
start = dt.datetime.strptime(start_date, '%Y-%m-%d')
end = dt.datetime.strptime(end_date, '%Y-%m-%d')
vacation_list = [start + dt.timedelta(days=x) for x in range((end-start).days)]
vacation_list
# Strip off the year and save a list of strings in the format %m-%d
vacation = [dt.datetime.strftime(vday,'%m-%d') for vday in vacation_list]
vacation

# Use the `daily_normals` function to calculate the normals for each date string 
# and append the results to a list called `normals`.
normals = [daily_normals(day) for day in vacation]
normals

# I like list comprehension :)

In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index

# it might not be the prettiest way to get this into a dataframe but it works

# creating dictionary for storing data
df_dict = {'tmin':[], 'tavg':[], 'tmax':[], 'date':[]}

# storing data into dictionary format
for row in normals:
    df_dict['tmin'].append(row[0][0])
    df_dict['tavg'].append(row[0][1])
    df_dict['tmax'].append(row[0][2])

# adding date part
for day in vacation:
    df_dict['date'].append(day)

# making a dataframe
df = pd.DataFrame(df_dict)

# setting date as index
df.set_index('date', inplace=True)

df

In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
df.plot.area(stacked=False, figsize=(10,5))
plt.ylabel('Temperature')
plt.show()

## Close Session

In [ ]:
session.close()